In [54]:
import requests
import json
import matplotlib as plt
from pprint import pprint
import pandas as pd
import numpy as np
import warnings

In [55]:
zip_crime_csv = "Resources/police_data.csv"
zip_crime_reader = pd.read_csv(zip_crime_csv, encoding="ISO-8859-1")
zip_crime_reader.head(2)

,Incident Number w/year,Year of Incident,Watch,Call (911) Problem,Type of Incident,Date1 of Occurrence,Month1 of Occurence,Day1 of the Week,Time1 of Occurrence,Call Dispatch Date Time,X Coordinate,Y Cordinate,Zip Code,City,State
0,038164-2018,2018,3,6XEA - DISTURBANCE EMERG AMB,RESIST ARREST SEARCH OR TRANSPORT,2/21/2018,February,Wed,19:30,2/21/2018 19:41,NaN,NaN,NaN,NaN,NaN
1,066284-2018,2018,1,6X - MAJOR DIST (VIOLENCE),ASSAULT -PUB SERV (PEACE OFFICER/JUDGE),3/30/2018,March,Fri,01:40,3/30/2018 01:27,NaN,NaN,NaN,NaN,NaN


In [56]:
zip_crime_reader.count()

Incident Number w/year     133949
Year of Incident           133949
Watch                      133949
Call (911) Problem         132683
Type of Incident           133949
Date1 of Occurrence        133949
Month1 of Occurence        133949
Day1 of the Week           133949
Time1 of Occurrence        133949
Call Dispatch Date Time    132673
X Coordinate                98503
Y Cordinate                 98503
Zip Code                   133669
City                       132123
State                      132974
dtype: int64

In [57]:
zip_crime_dropped = zip_crime_reader.dropna(how='any')
zip_crime_dropped.count()

Incident Number w/year     95440
Year of Incident           95440
Watch                      95440
Call (911) Problem         95440
Type of Incident           95440
Date1 of Occurrence        95440
Month1 of Occurence        95440
Day1 of the Week           95440
Time1 of Occurrence        95440
Call Dispatch Date Time    95440
X Coordinate               95440
Y Cordinate                95440
Zip Code                   95440
City                       95440
State                      95440
dtype: int64

In [58]:
zip_crime_dropped.columns

Index(['Incident Number w/year', 'Year of Incident', 'Watch',
       'Call (911) Problem', 'Type of Incident', 'Date1 of Occurrence',
       'Month1 of Occurence', 'Day1 of the Week', 'Time1 of Occurrence',
       'Call Dispatch Date Time', 'X Coordinate', 'Y Cordinate', 'Zip Code',
       'City', 'State'],
      dtype='object')

In [59]:
zip_crime_dropped.dtypes

Incident Number w/year      object
Year of Incident             int64
Watch                        int64
Call (911) Problem          object
Type of Incident            object
Date1 of Occurrence         object
Month1 of Occurence         object
Day1 of the Week            object
Time1 of Occurrence         object
Call Dispatch Date Time     object
X Coordinate               float64
Y Cordinate                float64
Zip Code                   float64
City                        object
State                       object
dtype: object

In [60]:
zip_crime_df = zip_crime_dropped.loc[:, ["Zip Code", "X Coordinate", "Y Cordinate", "City", "Type of Incident"]]
zip_crime_renamed_df = zip_crime_df.rename(columns={"Zip Code": "Zip Code","X Coordinate": "Longitude", "Y Cordinate": "Latitude", "City": "City","Type of Incident":"Type of Crime"})
zip_crime_renamed_df.head(10)

,Zip Code,Longitude,Latitude,City,Type of Crime
205,75212.0,2476498.585,6971715.153,DALLAS,VIO PROTECT ORDER BIAS/PREJUDICE
212,75253.0,2548241.858,6932130.126,DALLAS,ABANDONED PROPERTY (NO OFFENSE)
219,75217.0,2525365.155,6960102.895,DALLAS,PUBLIC INTOXICATION
221,75216.0,2496126.907,6941940.628,DALLAS,ASSAULT -BODILY INJURY ONLY
222,75216.0,2487800.730,6937829.316,DALLAS,WARRANT-DALLAS PD (ALIAS/CAPIAS)
229,75211.0,2468877.713,6961189.651,DALLAS,UNAUTHORIZED USE OF MOTOR VEH - (ATT) TRUCK OR...
230,75204.0,2494114.098,6976578.343,DALLAS,"CRIM MISCHIEF >OR EQUAL $750 < $2,500"
231,75215.0,2492637.330,6968271.897,DALLAS,PUBLIC INTOXICATION
233,75217.0,2524662.487,6947002.514,DALLAS,FAIL TO ID -FUGITIVE FROM JUSTICE
234,75210.0,2510456.676,6966237.837,DALLAS,DWI 1 PREV CONV


In [61]:
unique_crimes = len(zip_crime_renamed_df["Type of Crime"].unique())
unique_zip = len(zip_crime_renamed_df["Zip Code"].unique())
unique_zipcrimes = pd.DataFrame({"Number of Unique ZipCodes": [unique_zip] , "Number of Unique Crimes": [unique_crimes] })
unique_zipcrimes

,Number of Unique ZipCodes,Number of Unique Crimes
0,81,549


In [76]:
#warnings.simplefilter(action='ignore', category=FutureWarning)
#print('x' in np.arange(5))   #returns False, without Warning

zip_city = zip_crime_renamed_df["City"].value_counts()
zip_city
#zip_crime_loc = zip_crime_renamed_df.loc[zip_crime_renamed_df["City"] == "Garland"]
#zip_crime_loc


#zip_crime_loc = zip_crime_renamed_df.loc[(zip_crime_renamed_df["Zip Code"] == "PUBLIC INTOXICATION") | (zip_crime_renamed_df["Zip Code"] == "ASSAULT -BODILY INJURY ONLY"),:]
#print(zip_crime_loc)

DALLAS             94902
Dallas               227
GARLAND               46
DAL                   32
MESQUITE              30
RICHARDSON            26
DUNCANVILLE           26
DESOTO                21
DALLLAS               19
CARROLLTON            18
ROWLETT               17
DLS                   12
IRVING                 8
LANCASTER              7
DALAS                  7
COPPELL                6
PLANO                  4
GRAND PRAIRIE          4
DALLS                  4
GRAND PRAI             3
CEDAR HILL             3
UNIVERSITY PARK        3
SEAGOVILLE             3
ADDISON                3
DALLAS TX              1
ALICE                  1
DALALS                 1
DA                     1
DUNCANVILL             1
RED OAK                1
DALLASS                1
FARMERSBRA             1
DALALAS                1
Name: City, dtype: int64

In [83]:
zip_crime_changed_df = zip_crime_renamed_df.replace({"DAL": "Dallas", "DALLLAS": "Dallas","DALLLAS": "Dallas","DLS": "Dallas","DALAS": "Dallas","DALLS": "Dallas","DALLAS TX": "Dallas","DALALS": "Dallas","DA": "Dallas","DALLASS": "Dallas","DALALAS": "Dallas"})
zip_crime_changed_df["City"].value_counts()

DALLAS             94902
Dallas               306
GARLAND               46
MESQUITE              30
DUNCANVILLE           26
RICHARDSON            26
DESOTO                21
CARROLLTON            18
ROWLETT               17
IRVING                 8
LANCASTER              7
COPPELL                6
GRAND PRAIRIE          4
PLANO                  4
ADDISON                3
CEDAR HILL             3
SEAGOVILLE             3
GRAND PRAI             3
UNIVERSITY PARK        3
FARMERSBRA             1
RED OAK                1
ALICE                  1
DUNCANVILL             1
Name: City, dtype: int64

In [86]:
dallas_crime_df = zip_crime_renamed_df.loc[no_null_crime_df["City"] == "Dallas"]
rowlett_crime_df = zip_crime_renamed_df.loc[no_null_crime_df["City"] == "ROWLETT"]
plano_crime_df = zip_crime_renamed_df.loc[no_null_crime_df["City"] == "PLANO"]
carrollton_crime_df = zip_crime_renamed_df.loc[no_null_crime_df["City"] == "CARROLLTON"]


#Create DataFrame with header City/Total Crimes in 2018/percent within the entire metroplex
#Bar / Line / Heat?
dallas_crime_df
#rowlett_crime_df
#plano_crime_df
#carrollton_crime_df
#len(dallas_crime)
#len()

,Zip Code,Longitude,Latitude,City,Type of Crime
1421,75236.0,2460273.113,6940008.440,Dallas,CRIM MISCHIEF > OR EQUAL $100 < $750
3696,75254.0,2488799.002,7030064.614,Dallas,BMV
4161,75243.0,2513871.328,7015007.431,Dallas,UNAUTHORIZED USE OF MOTOR VEH - AUTOMOBILE
4490,75212.0,2475000.211,6970406.942,Dallas,CRIMINAL TRESPASS AFFIDAVIT
5146,75236.0,2446440.236,6930894.406,Dallas,UNAUTHORIZED USE OF MOTOR VEH - AUTOMOBILE
5304,75217.0,2529362.269,6954832.768,Dallas,RECOVERED OUT OF TOWN STOLEN VEHICLE (NO OFFENSE)
5315,75212.0,2473436.681,6975189.112,Dallas,TRAF VIO -DUTY ON STRIKE FIX/HWY LANDSCAPE > O...
5549,75243.0,2516919.033,7022239.802,Dallas,BURGLARY OF HABITATION -NO FORCED ENTRY
5645,75217.0,2529362.269,6954832.768,Dallas,"THEFT OF PROP > OR EQUAL $2,500 <$30K (NOT SHO..."
6338,75204.0,2495903.744,6975818.116,Dallas,DWI 2 OR MORE PREV CONV
